In [1]:
import statsmodels.stats.power as smp

Statsmodels is a Python module that provides functionality for conducting many statistical tests and analyses. It has been tested against R and other statistical packages, and implements R-style formulas with pandas dataframes or numpy functions to fit models.

# **1. CALCULATING SAMPLE SIZE FOR A 2 INDEPENDENT SAMPLE T-TEST**

R-code version: 
---------------------------
    install.packages("pwr")
    require(pwr)
    
    samplesize.ttest <- function(baseline.mean, baseline.SD, pct.change) {
  if (missing(baseline.mean))
    stop("Need to specify baseline mean of your metric for sample size calculations. Use AVG in redshift.")
  if (missing(baseline.SD))
    stop("Need to specify baseline SD of your metric for sample size calculations; use STDDEV_POP in redshift")
  if (missing(pct.change))
    stop("Need to specify expected % change, bounded ]-1,1[, of your metric for sample size calculations.")
  if(abs(pct.change) >= 1)
    stop("pct.change must be between ]-1, 1[")
  
  new.mean <- baseline.mean*(pct.change + 1)
 
  effect_size = abs(new.mean - baseline.mean) / (baseline.SD)
  result = suppressWarnings(pwr.t.test(n=NULL, d = effect_size, sig.level = 0.05, power = 0.8, type = "two.sample"))
  print(result)
  
  message("You will require ", result$n, " users per experiment group")
  }

In [2]:
# EG: baseline mean is 1.5, SD is 3, and i want a minimum sample size neessary to detect a 10% increase:
# samplesize.ttest(1.5,3,0.1) --answer: 6280.049 per group

1

good resource:
1. https://scientificallysound.org/2017/07/20/python-calculating-sample-size-for-a-2-independent-sample-t-test/  
2. https://www.youtube.com/watch?v=tTgouKMz-eI （cohen effect size)

-- SQL query to get mean and stdev

select partner_id
, COUNT( unique_id)
, STDDEV_POP(LT)
, AVG(LT)
FROM 
(select 
partner_id
, unique_id
, sum(listening_time::float/3600) as LT
FROM f_device_activity_plus
where partner_id in ( 500, 777)
and device_create_date between '2016-08-16' and '2016-08-23'
and activity_date between '2016-08-16' and '2016-08-30'
group by 1,2
)
group by 1

In [1]:
from statsmodels.stats.power import tt_ind_solve_power


####enter parameters below for power analysis###############


baseline_mean = 212   # baseline mean
baseline_sd = 526     # baseline's standard deviation
pct_change = 0.05      # pct relative change that you expected


############################################################

######Power analysis(no need to change below)###############

############################################################

def power_analysis_t_test(baseline_mean = baseline_mean, baseline_sd = baseline_sd, pct_change = pct_change):
    new_mean = baseline_mean*(1 + pct_change)
    mean_diff = (abs(new_mean - baseline_mean))
    sd_diff = baseline_sd
    std_effect_size = mean_diff / sd_diff
    power = 0.8
    alpha = 0.05
    n = tt_ind_solve_power(effect_size=std_effect_size, 
                           alpha = alpha, power = power, ratio=1, alternative='two-sided')
    start = "\033[1m"
    end = "\033[0;0m"
    print ('You will require %s users per experiment group' % (start + str(n) + end))
    print ('This is each group''s sample size, not total')

power_analysis_t_test(baseline_mean = baseline_mean, baseline_sd = baseline_sd, pct_change = pct_change)

You will require 38655.1320181 users per experiment group
This is each groups sample size, not total


In [ ]:
# 23098 per group 

# ** 2. CALCULATING SAMPLE SIZE FOR  2 proportions **

good source:
 1. - https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/PASS/Tests_for_Two_Proportions_using_Effect_Size.pdf
 2. - http://jpktd.blogspot.com/2013/03/statistical-power-in-statsmodels.html
 3. - https://www.rdocumentation.org/packages/pwr/versions/1.2-0/topics/ES.h

    
As stated above, the effect size h is given by ℎ = 𝜑𝜑1 − 𝜑𝜑2. Cohen (1988) proposed the following interpretation
of the h values.
 - An h near 0.2 is a small effect, 
 - an h near 0.5 is a medium effect,
 - and an h near 0.8 is a large effect.
These values for small, medium, and large effects are popular in the social sciences. 

R-code version: 
---------------------------

samplesize.proptest <- function(baseline.pct, pct.change) {
  if (missing(baseline.pct))
    stop("Need to specify baseline pct of your metric for sample size calculations.")
  if (missing(pct.change))
    stop("Need to specify expected % change, bounded ]-1,1[, of your metric for sample size calculations.")
  if(abs(pct.change) >= 1)
    stop("pct.change must be between ]-1, 1[")
  if( (baseline.pct >= 1) | (baseline.pct <= 0) )
    stop("baseline.pct must be between ]0, 1[")
   
  new.pct <- baseline.pct*(pct.change + 1)
 
  result = pwr.2p.test(h = ES.h(new.pct, baseline.pct),n=NULL , sig.level = 0.05, power=0.8)
   
  print(result)
  message("You will require ", result$n, " users per experiment group")
}

In [ ]:
# EG: baseline pct is 30%, and i want minimum sample size necessary to detect a 5% drop:
# samplesize.proptest(0.3,-0.05) # You will require 14435.8539279507 users per experiment group --using R

In [32]:
smp.NormalIndPower().solve_power(effect_size = 0.2, nobs1=1600, alpha=0.01, power=0.9, ratio=None, alternative='two-sided')

0.3029142044215257

In [17]:
# gives you ratio
smp.NormalIndPower().solve_power(effect_size= 0.2, nobs1=1600, alpha=0.01, power=0.9, ratio=None, alternative='larger')

0.2553188921855437

In [ ]:
# calculate effect size for proportions

In [ ]:
3.12%

In [1]:
import math
import statsmodels.stats.power as smp
import numpy as np

####enter parameters below for power analysis###############

p1 = 0.25   # baseline proportion(rate)
pct_change = 0.05 # pct relative change that you expected to see



############################################################

######Power analysis(no need to change below)###############

############################################################

def power_analysis_proportion(p1 = p1, pct_change = pct_change):
    # calculate the p2
    p2 = p1*(1+ pct_change)
    # calculate effect_size
    effect_size = 2*math.asin(np.sqrt(p1))-2*math.asin(np.sqrt(p2))
    # power analysis
    n = smp.NormalIndPower().solve_power(effect_size= effect_size, nobs1 = None,
                                      alpha=0.05, power = 0.8, alternative='two-sided')
    start = "\033[1m"
    end = "\033[0;0m"
    print ('You will require %s users per experiment group' % (start + str(n) + end))
    print ('This is each group''s sample size, not total')

power_analysis_proportion(p1 = p1, pct_change = pct_change)

You will require 19144.7632661 users per experiment group
This is each groups sample size, not total


1.2948777007679233

1.0925581245225775

12.486271575113076

3.1948736116423557

# More detailed notes

 -- **1) #change two-sided power analysis to one-side (smaller or larger), sample size will decrease(see example below) **

In [18]:
smp.NormalIndPower().solve_power(effect_size= 0.03297593, nobs1 = None,
                                  alpha=0.05, power = 0.8, alternative='two-sided')

14435.85467632959

In [22]:
smp.NormalIndPower().solve_power(effect_size= -0.03297593, nobs1 = None,
                                  alpha=0.05, power = 0.8, alternative='smaller')

11371.140769144427

--** 2) if two sample size is not equal (ratio != 1), then it will get nobs1 (first sample size), use ratio to get second group size **

In [35]:
smp.NormalIndPower().solve_power(effect_size= 0.03297593, nobs1 = None, ratio = 2,
                                  alpha=0.05, power = 0.8, alternative='two-sided')
# this is nobs1

10826.891007281689

In [37]:
n1 = 10826.891007281689
ratio = 2
n2 = ratio *n1 # sample size for second group
total = n1 + n2
total

32480.67302184507

--** 3) solve power based on experiment result **

In [3]:
# this is for two sample t test
smp.TTestIndPower().solve_power(effect_size = 2/2.8, nobs1=300, ratio=1, alpha=0.05, alternative='two-sided')


1.0

In [5]:
# this is for two proportion test
smp.NormalIndPower().solve_power(effect_size= 0.2, nobs1=1600, alpha=0.01, power=None, ratio=1, alternative='larger')

0.99956655910784831

In [ ]:
# kim's experiment 566

In [ ]:
2005: Reg Wall	1.0	1.1%	0.0%	1640	
2006: No Reg Wall	0.3968	1.43%	29.93%	1683

In [14]:
import math
import statsmodels.stats.power as smp
import numpy as np

# two proportion (subscriber rate)
p1 = 1.1*0.01
p2 = 1.43*0.01

# calculate effect_size
effect_size = 2*math.asin(np.sqrt(p1))-2*math.asin(np.sqrt(p2))
nobs1 = 1640
nobs2 = 1683
ratio = nobs2/nobs1

result_power = smp.NormalIndPower().solve_power(effect_size= effect_size, nobs1=nobs1, alpha=0.05, 
                                 power=None, ratio=ratio, alternative='two-sided')

print(' power result: %.9f'% result_power)


 power result: 0.135434538


In [ ]:
# listening minutes

In [ ]:
ab_test_name	ab_test_id	pctl_listening_min	std_ev
Reg Wall	2005	36.27936991869919	81.16451374548281
No Reg Wall	2006	37.88743315508022	84.18743984166845

In [20]:
# listening minutes 
baseline_mean = 36.2793699
new_mean = 37.8874332
mean_diff = (abs(new_mean - baseline_mean))
baseline_sd = 81.1645
new_sd = 84.1874
sd_diff = (baseline_sd + new_sd)/2
std_effect_size = mean_diff / sd_diff

nobs1 = 1640
nobs2 = 1683
ratio = nobs2/nobs1

alpha = 0.05


power_result = smp.tt_ind_solve_power(effect_size=std_effect_size, nobs1 = nobs1,
                           alpha = alpha, power = None, 
                           ratio=ratio, alternative='two-sided')
print ('power result: %.5f'%power_result)

power result: 0.08621
